这个记事本是用来搭建CRNN网络的，用来做recognition

In [9]:
import os
import time
import torch
import torchvision
from torch import nn
from torchvision import models
from torchvision import transforms
from utils import *
from matplotlib import pyplot as plt
import numpy as np
from torch.utils import tensorboard as tb

In [10]:
# # 测试用的代码，不用看（打印出神经网络的结构）
# model = models.resnet50(pretrained=True)
# # model_cnn = nn.Sequential(*list(model.children())[:-1])
# for chi in model.children():
#     print(chi)
#     print('-------------------')

这里用resnet18除去最后的fc作为cnn的部分，lstm作为rnn的部分。<br>
输入1x128x128图片<br>
经过cnn部分，先是卷到了512x4x4，然后经过平均池化层变成512x1x1<br>
然后展平，经过线性变换放入lstm的hidden和cell

In [11]:
class CRNN(nn.Module):
    """
    这是一个基于CNN和LSTM的CRNN模型，用于生成文本

    """
    def __init__(self, num_classes=128, hidden_dim=1024, io_dim=1024, num_layers=4, bidirectional=True, device='cuda:0'):
        """
        args:
            num_classes: ...
            hidden_dim: int, the dimension of the hidden state of the LSTM
            io_dim: int, the dimension of the input and output of the LSTM
            num_layers: int, the number of layers of the LSTM
            bidirectional: bool, whether to use bidirectional LSTM
            device: str, the device
        """
        super(CRNN, self).__init__()
        self.direction_factor = 2 if bidirectional else 1
        self.num_layers = num_layers
        # num-classes对应ascii码表的128种字符
        self.num_classes = num_classes
        # hidden_dim是LSTM的隐藏层（hidden state）和细胞状态（cell state）的维度
        self.hidden_dim = hidden_dim
        # io_dim是LSTM的输入和输出的维度
        self.io_dim = io_dim
        self.device = device
        # max num of characters of the generated text
        self.max_len = 64
        # 1x1卷积层，用于将灰度图转换为3通道图像以适应ResNet的输入
        self.conv1 = nn.Conv2d(1, 3, 1)
        # 使用ResNet50作为CNN的基础模型，去掉最后一层全连接层
        self.cnn = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.cnn = nn.Sequential(*list(self.cnn.children())[:-1])  # output dim is 2048
        # LSTM层，输入维度为io_dim，隐藏层维度为hidden_dim
        self.rnn = nn.LSTM(io_dim, hidden_dim, num_layers, batch_first=True, bidirectional=bidirectional)
        # 将CNN的输出转换为LSTM的隐藏状态和细胞状态
        self.h0_fc = nn.Linear(2048, hidden_dim * num_layers * self.direction_factor)
        self.c0_fc = nn.Linear(2048, hidden_dim * num_layers * self.direction_factor)
        # 将LSTM的输出转换为最终的输出，即字符概率分布
        self.out_fc = nn.Linear(hidden_dim * self.direction_factor, num_classes)
        # 将字符的索引转换为字符的embedding
        self.embedding = nn.Embedding(num_classes, io_dim)
        # dropout防止过拟合
        self.dropout = nn.Dropout(0.75)
        self.to(device)
    
    def init_state(self, img):
        # 通过CNN卷出 lstm 的 hidden state 和 cell state
        x = self.conv1(img)         # batch_size, 3, 64, 64
        x = self.cnn(x)             # batch_size, 512, 1, 1
        x = x.view(x.size(0), -1)   # batch_size, 512
        x = x.unsqueeze(0)          # 1, batch_size, 512
        h0 = self.h0_fc(x)          # 1, batch_size, hidden_dim
        c0 = self.c0_fc(x)          # 1, batch_size, hidden_dim
        h0 = h0.view(-1, self.hidden_dim, self.num_layers * self.direction_factor).permute(2, 0, 1).contiguous()  # 4, batch_size, hidden_dim
        c0 = c0.view(-1, self.hidden_dim, self.num_layers * self.direction_factor).permute(2, 0, 1).contiguous()  # 4, batch_size, hidden_dim
        return h0, c0
    
    def next_char(self, x, h_c_n):
        # print("next char x shape: ", x.shape)
        h_n, c_n = h_c_n
        # x: the embedding of the last character
        # h_n: the hidden state of the last character
        # c_n: the cell state of the last character
        x, (h_n, c_n) = self.rnn(x, (h_n, c_n))
        # print("next char rnn output x shape: ", x.shape)
        x = self.out_fc(x)
        # x = self.dropout(x)
        # print("next char output x shape: ", x.shape)
        return x, (h_n, c_n)
    
    def forward(self, img):
        batch_size = img.size(0)
        h0, c0 = self.init_state(img)
        x = 2  # the index of the start token
        x = torch.tensor([x] * batch_size, dtype=torch.long).view(batch_size, 1).to(self.device)
        x = self.embedding(x)
        # print("after embedding x shape: ", x.shape)
        h_c_n = (h0, c0)
        temp = torch.zeros(batch_size, 1, self.num_classes).to(self.device)
        temp[:, 0, 2] = 1
        output = [temp]
        for i in range(1, self.max_len):
            x, h_c_n = self.next_char(x, h_c_n)
            output.append(x)
            x = x.argmax(dim=-1)
            x = self.embedding(x)
        output = torch.cat(output, dim=1)
        return output

In [12]:
# 加载数据集
dataset_IAM = RecDataset("IAM", "train")
dataset_CVL = RecDataset("CVL", "train")
dataset = torch.utils.data.ConcatDataset([dataset_IAM, dataset_CVL])
train_loader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=16)

In [13]:
# 测试CRNN进行预测时的形状是否符合要求

crnn = CRNN()
for step, (img, label) in enumerate(train_loader):
    img, label = img.to(crnn.device), label.to(crnn.device)
    print(img.shape)
    output = crnn(img)
    print(output.shape)
    break

torch.Size([10, 1, 128, 128])
torch.Size([10, 64, 128])


In [14]:
# 测试CRNN进行训练时的形状是否符合要求

crnn = CRNN()
for step, (img, label) in enumerate(train_loader):
    img, label = img.to(crnn.device), label.to(crnn.device)
    print(img.shape)
    h_n, c_n = crnn.init_state(img)
    print(h_n.shape, c_n.shape)
    break

torch.Size([10, 1, 128, 128])
torch.Size([8, 10, 1024]) torch.Size([8, 10, 1024])


In [15]:
def get_model_name(epoch, name):
    return f"{name}_{epoch}.pth"


def get_val_loss(model, dataloader, device):
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    model.eval()
    loss = 0
    with torch.no_grad():
        for step, (img, target) in enumerate(dataloader):
            img, target = img.to(device), target.to(device)
            h_n, c_n = model.init_state(img)
            h_n, c_n = h_n.to(device), c_n.to(device)
            output = torch.zeros(target.shape[0], model.max_len, model.num_classes).to(device)
            x = target[:, 0]
            output[:, 0] = x
            x = model.embedding(x.long().unsqueeze(1))
            for i in range(model.max_len-1):
                x = model.embedding(target[:, i].argmax(-1).unsqueeze(1))
                x, (h_n, c_n) = model.next_char(x, (h_n, c_n))
                output[:, i+1] = x.squeeze(1)
            loss += criterion(output.view(-1, model.num_classes), target.argmax(-1).view(-1))
    return loss / len(dataloader)

In [16]:
# 下面是训练的代码，使用教师强制训练
def train_crnn(model, lr, epochs, start_epoch, name, device='cuda:0'):
    """
    args:
        model: CRNN, the model to be trained
        lr: float, the learning rate
        epochs: int, the number of epochs to train
        device: str, the device to use
        start_epoch: int, the epoch to start training
        name: str, the name of the model or the task
    """

    # 如果start_epoch不为0，说明是从某个epoch开始训练的，需要加载模型
    if start_epoch:
        model_name = get_model_name(start_epoch, name)
        model_path = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/CRNN/"
        model.load_state_dict(torch.load(model_path + model_name))
        start_epoch += 1
        print(f"Model loaded from {model_name}")

    writer = tb.SummaryWriter('/root/tf-logs')  # tensorboard writer，可以在浏览器中查看训练过程
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    model.to(device)
    t0 = time.time()
    for epoch in range(start_epoch, epochs):
        model.train()
        for step, (img, target) in enumerate(train_loader):
            img, target = img.to(device), target.to(device)
            optimizer.zero_grad()
            loss = 0
            # 准备初始状态
            h_n, c_n = model.init_state(img)
            h_n, c_n = h_n.to(device), c_n.to(device)
            output = torch.zeros(target.shape[0], model.max_len, model.num_classes).to(device)
            x = target[:, 0]
            output[:, 0] = x
            x = model.embedding(x.long().unsqueeze(1))
            for i in range(model.max_len-1):
                x = model.embedding(target[:, i].argmax(-1).unsqueeze(1))
                x, (h_n, c_n) = model.next_char(x, (h_n, c_n))
                output[:, i+1] = x.squeeze(1)
            loss = criterion(output.view(-1, model.num_classes), target.argmax(-1).view(-1))
            loss.backward()
            # 将cnn的梯度调小，防止过拟合
            for param in model.cnn.parameters():
                param.grad *= 0.1
            optimizer.step()
            if time.time() - t0 > 1:
                # 每秒打印一次训练信息
                t0 = time.time()
                print(f"\rEpoch {epoch}, Iter {step}, Loss {loss.item():.4f}", end="")
                writer.add_scalar(f'{name}/Train_Loss', loss.item(), epoch * len(train_loader) + step)
        
        # 计算验证集上的loss
        val_loss = get_val_loss(model, val_loader, device)
        print(" " * 100, end="")
        print(f"\rEpoch {epoch}, Val Loss {val_loss.item()}", end="")
        writer.add_scalar(f'{name}/Val_Loss', val_loss.item(), epoch)

        model_name = get_model_name(epoch, "CRNN")
        model_path = "/root/autodl-tmp/APS360_Project/Machine_Learning_Output/CRNN/"
        torch.save(model.state_dict(), model_path + model_name)
        print(f"\nModel saved as {model_name}")


# 数据增广 & 加载数据集
trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(size=(128, 128), scale=(0.9, 1.2), ratio=(0.9, 1.1)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(10),
    transforms.ToTensor(),
])
val_trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

train_set_IAM = RecDataset("IAM", "train", trans)
train_set_CVL = RecDataset("CVL", "train", trans)
train_set = torch.utils.data.ConcatDataset([train_set_IAM, train_set_CVL])
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=16)

val_set_IAM = RecDataset("IAM", "val", val_trans)
val_set_CVL = RecDataset("CVL", "val", val_trans)
val_set = torch.utils.data.ConcatDataset([val_set_IAM, val_set_CVL])
val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=16)

# 创建模型
model = CRNN()

# 开始训练
train_crnn(model, lr=0.01, epochs=500, start_epoch=0, name="CRNN")

Epoch 0, Val Loss 2.3464198112487793                                                                                               
Model saved as CRNN_0.pth
Epoch 1, Iter 1228, Loss 2.0646